In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
# Pkg.add("CUDA")
# Pkg.add("Glob")
# Pkg.add("Images")
# Pkg.add("FastAI")
# Pkg.add("FastVision")
# Pkg.add("Flux")
# Pkg.add("Metalhead")
# Pkg.add("Images")
# Pkg.add("DataAugmentation")
# Pkg.add("CairoMakie")
# Pkg.add("StaticArrays")
# Pkg.add("MLDataPattern")
# Pkg.add("FastMakie")

  Activating project at `g:\桌面\2022 Fall\cs284A\Project`


In [2]:
using Glob
using CUDA
using FastAI
using FastVision
using MLDataPattern
using DataAugmentation, Images
using Flux
using Metalhead
using CairoMakie
using StaticArrays
using DICOM
using Images
using Printf
using NIfTI
using Base.Iterators
using Plots
CairoMakie.activate!(type="png")

In [3]:
mask_dir = raw"D:\RSNA-2022-dataset\segmentations"
train_image_dir = raw"G:\桌面\2022 Fall\cs284A\Project\Compressed_data\train_images"

"G:\\桌面\\2022 Fall\\cs284A\\Project\\Compressed_data\\train_images"

In [4]:
mask_names = []
for f in readdir(mask_dir, join=false)
    push!(mask_names, splitext(f)[1])
end

In [5]:
matched_names = []
ct = 0
for img in readdir(train_image_dir, join=false)
    f = splitext(splitext(img)[1])[1]
    if f in mask_names
        push!(matched_names, f)
        ct += 1
    end
end
println("Found $ct of $(size(mask_names)[1])")

Found 87 of 87


In [6]:
for mask in mask_names
    if !(mask in matched_names)
        println(mask)
    end
end

In [7]:
image_size = (128, 128, 128)

(128, 128, 128)

In [8]:
container_images = Array{Float32, 4}(undef, image_size..., ct)
container_masks = Array{Int64, 4}(undef, image_size..., ct);

In [9]:
Threads.@threads for i = 1 : 87
    curr_path = train_image_dir * "\\" * matched_names[i] * ".img.txt"
    temp = zeros(128, 128, 128)
    read!(curr_path, temp)
    container_images[:,:,:, i] = deepcopy(temp)
end

In [10]:
Threads.@threads for i = 1 : 87
    curr_path = "G:\\桌面\\2022 Fall\\cs284A\\Project\\Compressed_data\\segmentations\\" * matched_names[i] * ".nii.txt"
    temp = zeros(128, 128, 128)
    read!(curr_path, temp)
    temp = reinterpret(Int64, temp)
    container_masks[:,:,:, i] = deepcopy(temp)
end

In [11]:
typeof(container_images)

Array{Float32, 4}

In [12]:
typeof(container_masks)

Array{Int64, 4}

In [13]:
data = (container_images, container_masks);

In [14]:
maximum(container_images), minimum(container_images)

(1.0f0, 0.0f0)

In [15]:
maximum(container_masks), minimum(container_masks)

(8, 1)

In [16]:
data_mean, data_std = FastVision.imagedatasetstats(container_images, Gray{N0f8}) 
data_mean, data_std = StaticArraysCore.SVector{1, Float32}(data_mean[1]), StaticArraysCore.SVector{1, Float32}(data_std[1])

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


(Float32[0.2795012], Float32[0.16293259])

In [17]:
taskk = SupervisedTask(
    (FastVision.Image{3}(), Mask{3}(1:8)), # 7 classes and 1 background
    (
        FastVision.ProjectiveTransforms((image_size)),
        ImagePreprocessing(means = data_mean, stds = data_std, C = Gray{N0f8}),
        FastAI.OneHot()
    )
)

SupervisedTask(FastVision.Image{3} -> Mask{3, Int64})

In [18]:
train_files, val_files = MLDataPattern.splitobs(data, 0.8);

In [19]:
traindl, validdl = taskdataloaders(train_files, val_files, taskk, 1);

In [20]:
conv = (stride, in, out) -> Conv((3, 3, 3), in=>out, stride=stride, pad=(1, 1, 1))
tran = (stride, in, out) -> ConvTranspose((4, 4, 4), in=>out, stride=stride, pad=1)

conv1 = (in, out) -> Chain(conv(1, in, out), BatchNorm(out), x -> leakyrelu.(x))
conv2 = (in, out) -> Chain(conv(2, in, out), BatchNorm(out), x -> leakyrelu.(x))
tran2 = (in, out) -> Chain(tran(2, in, out), BatchNorm(out), x -> leakyrelu.(x))

#15 (generic function with 1 method)

In [21]:
function unet3D(in_chs, lbl_chs)
    # Contracting layers
    l1 = Chain(conv1(in_chs, 4))
    l2 = Chain(l1, conv1(4, 4), conv2(4, 8))
    l3 = Chain(l2, conv1(8, 8), conv2(8, 16))
    l4 = Chain(l3, conv1(16, 16), conv2(16, 32))
    l5 = Chain(l4, conv1(32, 32), conv2(32, 64))

    # Expanding layers
    l6 = Chain(l5, tran2(64, 32), conv1(32, 32))
    l7 = Chain(Parallel(+, l6, l4), tran2(32, 16), conv1(16, 16))       # Residual connection between l6 & l4
    l8 = Chain(Parallel(+, l7, l3), tran2(16, 8), conv1(8, 8))       # Residual connection between l7 & l3
    l9 = Chain(Parallel(+, l8, l2), tran2(8, 4), conv1(4, 4))          # Residual connection between l8 & l2
    l10 = Chain(l9, conv1(4, lbl_chs))
end

unet3D (generic function with 1 method)

In [22]:
model_1 = unet3D(1, 8);
# taskk, model= loadtaskmodel("try2_stage1_5.jld2"); # continue train from prev saved model
model_1 = model_1 |> gpu;

In [23]:
lossfn = tasklossfn(taskk)

_segmentationloss (generic function with 1 method)

In [24]:
optimizer = Flux.ADAM() 

Adam(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [25]:
learner = Learner(model_1, lossfn; data=(traindl, validdl), optimizer, callbacks=[ToGPU()])
# learner = Learner(model_1, lossfn; data=(traindl, validdl), optimizer)

Learner()

In [26]:
fitonecycle!(learner, 1, 0.033)

: 

: 

In [ ]:
fitonecycle!(learner, 9, 0.033)

In [ ]:
savetaskmodel("try3_stage1_10.jld2", taskk, learner.model, force = true)

In [ ]:
for i = 2:25
    fitonecycle!(learner, 10, 0.033)
    savetaskmodel("try3_stage1_"*string(i*10)*".jld2", taskk, learner.model, force = true)
    flush(stdout)
end
    